# Final project: House Prices Prediction

<i>(last updated: 06/01/2022)</i>

<br>
<b>Participants:</b> &ensp;TRẦN ĐẠI CHÍ - 18127070

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;TRẦN XUÂN LỘC - 18127131

<b>Class:</b> 19KHDL<br>
<b>Teacher Assistant:</b> TRẦN TRUNG KIÊN <br>
---

## Environment

In [2]:
import sys
sys.executable

'C:\\Users\\USER\\anaconda3\\envs\\min_ds-env\\python.exe'

## Import

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as XGB

---

## Collecting data

### Introduction

Description: <cite>[1]</cite> "The Ames Housing dataset was compiled by Dean De Cock for use in data science education. It's an incredible alternative for data scientists looking for a modernized and expanded version of the often cited Boston Housing dataset." 
The dataset has 79 explanatory variables 1460 records describing (almost) every aspect of residential homes in Ames, Iowa.

Dataset: [House Prices - Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)

### Download data from kaggle

In [4]:
#!pip install xgboost

In [5]:
#!pip install -q kaggle

In [6]:
#!mkdir ~/.kaggle

In [7]:
#!cp kaggle.json ~/.kaggle/

In [8]:
#!chmod 600 ~/.kaggle/kaggle.json

In [9]:
#!kaggle competitions download -c house-prices-advanced-regression-techniques

---

## Explore data

### Load files

In [19]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
#print("Train size: ", train_df.shape)
#print("Test size: ", test_df.shape)
df = train_df
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### How many rows and columns?

In [20]:
rows, cols = df.shape
print("There are "+str(cols)+" features with "+str(rows)+" records")

There are 81 features with 1460 records


### What does each row mean?

- write here

### Are there any duplicated rows?

We check whether rows are duplicated, then save the result into `have_duplicated_rows`, if there are any rows duplicated the result is True and otherwise it is False.

In [21]:
have_duplicated_rows = df.duplicated().all()

### What does each column mean?

- Write here

### How many numerical columns?

In [35]:
num_df = df.select_dtypes(exclude='object')
print("There are " +str(num_df.shape[1])+ ' numerical columns.')
num_df.columns

There are 38 numerical columns.


Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

### Data distribution in numerical columns

In [31]:
def numerical_distribution(col):
    result = []
    result.append(col.isna().mean() * 100) # missing percpercentage
    result.append(col.min()) # min
    result.append(col.max()) # max
    return pd.Series(result, index=['missing_percentage', 'min', 'max'])
    
num_df.agg(numerical_distribution).round(1)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
missing_percentage,0.0,0.0,17.7,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,20.0,21.0,1300.0,1.0,1.0,1872.0,1950.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2006.0,34900.0
max,1460.0,190.0,313.0,215245.0,10.0,9.0,2010.0,2010.0,1600.0,5644.0,...,857.0,547.0,552.0,508.0,480.0,738.0,15500.0,12.0,2010.0,755000.0


### How many categorical columns?

In [36]:
cate_df = df.select_dtypes(include='object')
print("There are " +str(cate_df.shape[1])+ ' categorical columns.')
cate_df.columns

There are 43 categorical columns.


Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

### Data distribution in categorical columns

In [37]:
def categorical_distribution(col):
    result = []
    result.append((col.isna().mean() * 100).round(1))   # missing percentage
    
    num_values = col.str.split(';').explode().nunique()
    result.append(num_values) # num_values
    
    data = col.str.split(";").explode().value_counts()
    ans = data.keys().tolist()
    freqs = (data / data.sum() * 100).round(1).tolist()
    value_percentages = dict(zip(ans, freqs))
    result.append(value_percentages) # value_percentages
    
    return pd.Series(result, index=['missing_percentage', 'num_values', 'value_percentages'])


cate_df.agg(categorical_distribution).round(1)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
missing_percentage,0.0,0.0,93.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.5,5.5,5.5,5.5,0.0,99.5,80.8,96.3,0.0,0.0
num_values,5,2,2,4,4,2,5,3,25,9,...,6,3,5,5,3,3,4,4,9,6
value_percentages,"{'RL': 78.8, 'RM': 14.9, 'FV': 4.5, 'RH': 1.1,...","{'Pave': 99.6, 'Grvl': 0.4}","{'Grvl': 54.9, 'Pave': 45.1}","{'Reg': 63.4, 'IR1': 33.2, 'IR2': 2.8, 'IR3': ...","{'Lvl': 89.8, 'Bnk': 4.3, 'HLS': 3.4, 'Low': 2.5}","{'AllPub': 99.9, 'NoSeWa': 0.1}","{'Inside': 72.1, 'Corner': 18.0, 'CulDSac': 6....","{'Gtl': 94.7, 'Mod': 4.5, 'Sev': 0.9}","{'NAmes': 15.4, 'CollgCr': 10.3, 'OldTown': 7....","{'Norm': 86.3, 'Feedr': 5.5, 'Artery': 3.3, 'R...",...,"{'Attchd': 63.1, 'Detchd': 28.1, 'BuiltIn': 6....","{'Unf': 43.9, 'RFn': 30.6, 'Fin': 25.5}","{'TA': 95.1, 'Fa': 3.5, 'Gd': 1.0, 'Po': 0.2, ...","{'TA': 96.2, 'Fa': 2.5, 'Gd': 0.7, 'Po': 0.5, ...","{'Y': 91.8, 'N': 6.2, 'P': 2.1}","{'Gd': 42.9, 'Fa': 28.6, 'Ex': 28.6}","{'MnPrv': 55.9, 'GdPrv': 21.0, 'GdWo': 19.2, '...","{'Shed': 90.7, 'Othr': 3.7, 'Gar2': 3.7, 'TenC...","{'WD': 86.8, 'New': 8.4, 'COD': 2.9, 'ConLD': ...","{'Normal': 82.1, 'Partial': 8.6, 'Abnorml': 6...."


---

## Preprocess data

### NaN values

In [ ]:
#print total null values of each columns
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  
train_df.isnull().sum()

In [6]:
#use log1p to SalePrice column to avoid inf values from log(0)
train_df['SalePrice'] = np.log1p(train_df['SalePrice'])

In [7]:
Y_train = train_df['SalePrice']
#merge two table of the training and testing file to preprocess
all_data = pd.concat([train_df, test_df], axis = 0, sort = False)
all_data = all_data.drop(['Id', 'SalePrice'], axis = 1)

In [8]:
Total = all_data.isnull().sum().sort_values(ascending = False)
missing_data = pd.DataFrame(Total).rename(columns={0: "Total missing"})
missing_data.head()

,Total missing
PoolQC,2909
MiscFeature,2814
Alley,2721
Fence,2348
FireplaceQu,1420


In [9]:
#drop columns which have total missing data > 5 and columns with max total missing are 4
all_data.drop((missing_data[missing_data['Total missing'] > 5]).index, axis = 1, inplace = True)

In [10]:
all_data.isnull().sum().sort_values().tail()

Utilities       2
Functional      2
BsmtFullBath    2
BsmtHalfBath    2
MSZoning        4
dtype: int64

In [11]:
print('Categorical columns:', all_data.select_dtypes(include=['object']).columns)
print('Numeric columns:', all_data.select_dtypes(exclude=['object']).columns)

Categorical columns: Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')
Numeric columns: Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='o

In [13]:
#need check BsmtFullBath and BsmtHalfBath
numeric_missing_columns = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageArea', 'GarageCars']
for val in numeric_missing_columns:
    all_data[val] = all_data[val].fillna(all_data[val].mean)
categorical_missing_columns = ['Exterior1st', 'Exterior2nd', 'SaleType', 'MSZoning', 'Electrical', 'KitchenQual']                   
for val in categorical_missing_columns:
    all_data[val] = all_data[val].fillna(all_data[val].mode()[0])

In [14]:
all_data['Functional'] = all_data['Functional'].fillna('Typ')
#all values of Utilities column are the same, so we should delete it
all_data.drop(['Utilities'], axis=1, inplace=True) 
all_data.isnull().sum().max() #no more missing columns

0

In [15]:
numeric_features = all_data.dtypes[all_data.dtypes != 'object'].index
skewed_features = all_data[numeric_features].apply(lambda x: skew(x)).sort_values()
#get all columns with skewness > 0.5 and make log transformation on them
col_skewness = skewed_features[abs(skewed_features) > 0.5]
col_skewness

YearBuilt        -0.599806
OverallCond       0.570312
BsmtFullBath      0.624832
HalfBath          0.694566
Fireplaces        0.733495
TotRmsAbvGrd      0.758367
2ndFlrSF          0.861675
BsmtUnfSF         0.919339
TotalBsmtSF       1.156894
GrLivArea         1.269358
MSSubClass        1.375457
BsmtFinSF1        1.425230
1stFlrSF          1.469604
WoodDeckSF        1.842433
OpenPorchSF       2.535114
BsmtHalfBath      3.931594
ScreenPorch       3.946694
EnclosedPorch     4.003891
BsmtFinSF2        4.146143
KitchenAbvGr      4.302254
3SsnPorch        11.376065
LowQualFinSF     12.088761
LotArea          12.822431
PoolArea         16.898328
MiscVal          21.947195
dtype: float64

In [16]:
all_data['Total_Square_Feet'] = all_data['BsmtFinSF1'] + all_data['BsmtFinSF2'] + all_data['1stFlrSF'] + all_data['2ndFlrSF'] + all_data['TotalBsmtSF']
all_data['Total_Bath'] = all_data['FullBath'] + (0.5 * all_data['HalfBath']) + all_data['BsmtFullBath'] + (0.5 * all_data['BsmtHalfBath'])
#all_data['Total_Porch_Area'] = all_data['OpenPorchSF'] + all_data['3SsnPorch'] + all_data['EnclosedPorch'] + all_data['ScreenPorch'] + all_data['WoodDeckSF']
#all_data['SqFtPerRoom'] = all_data['GrLivArea'] / (all_data['TotRmsAbvGrd'] + all_data['FullBath'] + all_data['HalfBath'] + all_data['KitchenAbvGr'])

In [17]:
for val in col_skewness.index:
    all_data[val] = np.log1p(all_data[val])    

In [18]:
#convert categorical variable into dummy variables
all_data = pd.get_dummies(all_data) 
all_data.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Total_Square_Feet,Total_Bath,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.110874,9.042040,7,1.791759,7.602900,2003,6.561031,0.0,5.017280,6.753438,6.753438,6.751101,0.0,7.444833,0.693147,0.000000,2,0.693147,3,0.693147,2.197225,0.000000,2.0,548.0,0.000000,4.127134,0.000000,0.0,0.0,0.0,0.0,2,2008,3272.0,3.5,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1

## Ask some meaningful questions

In this section, we would  analyze our observations to answer some meaningful questions belows:
- **Question 1:**
- **Question 2:**
- **Question 3:**
- **Question 4:**

---

## Analyze data to answer the questions

### Question 1:

### Question 2:

### Question 3:

### Question 4:

---

## Expand Section

**Introduction about house prices problem.**
- write here

In [19]:
X_train = all_data[:len(Y_train)]
X_test = all_data[len(Y_train):]
print("Train size: ", X_train.shape)
print("Test size: ", X_test.shape)

Train size:  (1460, 220)
Test size:  (1459, 220)


In [20]:
the_model = XGB.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, random_state =7, nthread = -1, objective ='reg:squarederror')
the_model.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4603,
             enable_categorical=False, gamma=0.0468, gpu_id=-1,
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1.7817, missing=nan, monotone_constraints='()',
             n_estimators=2200, n_jobs=12, nthread=-1, num_parallel_tree=1,
             predictor='auto', random_state=7, reg_alpha=0.464,
             reg_lambda=0.8571, scale_pos_weight=1, subsample=0.5213,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
the_model.score(X_train, Y_train)

0.9595967030418314

In [22]:
#0.9584876666389691
#array([125978., 162983., 185833., ..., 170827., 121675., 221900.], dtype=float32)
#get above score for using all_data['Total_Square_Feet'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

#0.9591461897091682
#array([122688., 164253., 185703., ..., 169173., 122726., 226370.], dtype=float32)
#get above score for using:
#all_data['Total_Square_Feet'] = all_data['BsmtFinSF1'] + all_data['BsmtFinSF2'] + all_data['1stFlrSF'] + all_data['2ndFlrSF'] + all_data['TotalBsmtSF']
#all_data['Total_Bath'] = all_data['FullBath'] + (0.5 * all_data['HalfBath']) + all_data['BsmtFullBath'] + (0.5 * all_data['BsmtHalfBath'])

In [23]:
y_predict = np.floor(np.expm1(the_model.predict(X_test)))
y_predict

array([127390., 162944., 184540., ..., 169036., 122617., 225769.],
      dtype=float32)

---

### Commit our predictions

In [24]:
submission_file = pd.DataFrame({'Id':test_df['Id'], 'SalePrice':y_predict})
submission_file.to_csv('submission.csv', index=False)

**Kaggle** **score:** &ensp; <ins>0.12xx</ins>

---

## References

[1] Dean De Cock. (2016&asp; 08). House Prices - Advanced Regression Techniques. Retrieved January 01, 2022 from https://www.kaggle.com/c/house-prices-advanced-regression-techniques.

---